READING DATA FROM API AND WRITING TO DB

WEATHER

In [1]:
import openmeteo_requests
import requests
import pandas as pd
from pymongo import MongoClient
from bson import json_util
import json
import requests_cache
import pandas as pd
from retry_requests import retry

client = MongoClient("mongodb://%s:%s@127.0.0.1" % ("dap", "dap"))
db = client['TRY']
collection = db['weather1']
def check_data_in_db(data_dict, collection):
    # Check if the data already exists in the database
    existing_data = collection.find_one(data_dict)
    return existing_data is None
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 53.3331,
	"longitude": -6.2489,
	"start_date": "2023-01-01",
	"end_date": "2023-12-31",
	"hourly": ["temperature_2m", "relative_humidity_2m", "rain", "snowfall", "weather_code", "wind_speed_10m", "is_day", "sunshine_duration"],
	"timezone": "GMT"
}
responses = openmeteo.weather_api(url, params=params)
response = responses[0]
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(3).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(5).ValuesAsNumpy()
hourly_is_day = hourly.Variables(6).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["weather_code"] = hourly_weather_code
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["is_day"] = hourly_is_day
hourly_data["sunshine_duration"] = hourly_sunshine_duration
hourly_dataframe = pd.DataFrame(data = hourly_data)
data_dict = hourly_dataframe.to_dict('records')

for data in data_dict:

	if check_data_in_db(data,collection):
		collection.insert_one(data)
	else:
		print("Data already exists in the database.")




Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
D

AIR QUALITY

In [2]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
collection = db['Airquality1']
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": 53.3331,
	"longitude": -6.2489,
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "dust", "uv_index", "uv_index_clear_sky"],
	"start_date": "2023-01-01",
	"end_date": "2023-12-31"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
#print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
#print(f"Elevation {response.Elevation()} m asl")
#print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
#print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
hourly_nitrogen_dioxide = hourly.Variables(3).ValuesAsNumpy()
hourly_sulphur_dioxide = hourly.Variables(4).ValuesAsNumpy()
hourly_ozone = hourly.Variables(5).ValuesAsNumpy()
hourly_dust = hourly.Variables(6).ValuesAsNumpy()
hourly_uv_index = hourly.Variables(7).ValuesAsNumpy()
hourly_uv_index_clear_sky = hourly.Variables(8).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
hourly_data["ozone"] = hourly_ozone
hourly_data["dust"] = hourly_dust
hourly_data["uv_index"] = hourly_uv_index
hourly_data["uv_index_clear_sky"] = hourly_uv_index_clear_sky

hourly_dataframe = pd.DataFrame(data = hourly_data)
data_dict = hourly_dataframe.to_dict('records')
for data in data_dict:

	if check_data_in_db(data,collection):
		collection.insert_one(data)
	else:
		print("Data already exists in the database.")
#collection.insert_many(data_dict)
#print(hourly_dataframe)

Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
D

FOOTFALL

In [3]:
import pandas as pd
def read_data_from_csv(filename):
    try:
        df = pd.read_csv(filename)
        return df
    except Exception as e:
        print("An error occurred:", e)
        return None
csv_filename = "C://Users//alpho//Downloads//footfall.csv"
df = read_data_from_csv(csv_filename)
if df is None:
        print("Failed to read data from CSV file.")        
data_dict = df.to_dict(orient='records')
collection = db['footfall1']
for data in data_dict:

	if check_data_in_db(data,collection):
		collection.insert_one(data)
	else:
		print("Data already exists in the database.")

Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
Data already exists in the database.
D